In [1]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path
from contextlib import asynccontextmanager
import pandas as pd
# Load environment variables from .env file
load_dotenv()

# Configure LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "rag-tutor")

# Initialize LangSmith client (optional, but ensures connection)
try:
    from langsmith import Client
    langsmith_api_key = os.getenv("LANGCHAIN_API_KEY")
    if langsmith_api_key:
        langsmith_client = Client(api_key=langsmith_api_key)
        print("✅ LangSmith tracing enabled")
    else:
        print("⚠️ LANGCHAIN_API_KEY not set, tracing may not work")
except ImportError:
    print("⚠️ langsmith package not installed, tracing may not work")
except Exception as e:
    print(f"⚠️ LangSmith initialization warning: {e}")

# Add parent directory to path for imports
backend_dir = Path('/home/sofya/Documents/github/rag_tutor/app/backend').parent
app_dir = backend_dir.parent
sys.path.insert(0, str(Path('/home/sofya/Documents/github/rag_tutor/app/backend').parent))

from backend.agents.math_tutor_agent import MathTutorAgent



global agent
# Startup
try:
    # Determine paths - check if we're in Docker first
    if os.path.exists("/app"):
        # In Docker
        faiss_db_path = "/app/data/faiss_db"
        chunks_meta_path = "/app/data/all_chunks_with_meta_all.pickle"
    else:
        # Not in Docker, use relative paths
        project_root = Path('/home/sofya/Documents/github/rag_tutor')
        faiss_db_path = str(project_root / "data" / "faiss_db")
        chunks_meta_path = str(project_root / "data" / "all_chunks_with_meta_all.pickle")

    agent = MathTutorAgent(
        use_rag=os.getenv("USE_RAG", "true").lower() == "true",
        faiss_db_path=faiss_db_path,
        chunks_meta_path=chunks_meta_path
    )
    print("✅ Math Tutor Agent initialized successfully")
except Exception as e:
    print(f"❌ Failed to initialize agent: {e}")
    raise

agent = None
print("👋 Math Tutor Agent shut down")

# Global agent instance
agent: MathTutorAgent = None

agent = MathTutorAgent(
            use_rag=os.getenv("USE_RAG", "true").lower() == "true",
            faiss_db_path=faiss_db_path,
            chunks_meta_path=chunks_meta_path
        )


✅ LangSmith tracing enabled
✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from /home/sofya/Documents/github/rag_tutor/data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from /home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle
✅ Math Tutor Agent initialized successfully
👋 Math Tutor Agent shut down
✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from /home/sofya/Documents/github/rag_tutor/data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from /home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle


In [2]:
import backend
from  importlib import reload
reload(backend)
from backend.agents.math_tutor_agent import MathTutorAgent


agent = MathTutorAgent(
            use_rag=os.getenv("USE_RAG", "true").lower() == "true",
            faiss_db_path=faiss_db_path,
            chunks_meta_path=chunks_meta_path
        )

✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from /home/sofya/Documents/github/rag_tutor/data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from /home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle


In [3]:
import json
with open('validation_all.json', 'r', encoding='utf-8') as f:     
    validation_data_all = json.loads(f.read())

In [4]:
validation_data_all[0]

{'query': 'Что такое первообразная и как она связана с неопределённым интегралом?',
 'target_level': 'university',
 'relevant_chunk_ids': [5, 6, 7],
 'source_file': 'university/2_course/Математический анализ. Интегральное исчисление_Виленкин, Куницкая, Мордкович_1979 -177с.djvu',
 'level': 'university',
 'book_title': 'Математический анализ. Интегральное исчисление_Виленкин, Куницкая, Мордкович_1979 -177с',
 'grade': '2_course',
 'expected_sources': 'Виленкин, Куницкая, Мордкович',
 'reference_answer': "Первообразной для функции \\( f(x) \\) на промежутке \\( X \\) называется функция \\( F(x) \\), такая что \\( F'(x) = f(x) \\) для всех \\( x \\in X \\). Множество всех первообразных функции \\( f(x) \\) называется её неопределённым интегралом и обозначается \\( \\int f(x)\\,dx = F(x) + C \\), где \\( C \\) — произвольная постоянная. "}

In [5]:
agent.__dict__.keys()

dict_keys(['code_interpreter', 'last_code_result', 'use_rag', 'rag_retriever', 'llm', 'system_prompt', 'tools', 'agent'])

## Оценка качества retriever

In [7]:
import math
from typing import List, Dict, Any

def validate_retriever(retriever, val_set: List[Dict[str, Any]], k_list: List[int] = [5, 10, 20, 30, 50]):
    results_all = []
    results_val_rag = []

    def dcg_at_k(relevance_scores: List[int], k: int) -> float:
        """Discounted Cumulative Gain @k"""
        return sum((rel / math.log2(i + 2)) for i, rel in enumerate(relevance_scores[:k]))

    def ndcg_at_k(relevance_scores: List[int], k: int) -> float:
        """Normalized DCG @k"""
        if not any(relevance_scores):
            return 1.0  # если нет релевантных, NDCG = 1 по соглашению
        dcg = dcg_at_k(relevance_scores, k)
        # Ideal DCG: все релевантные в начале
        ideal_relevance = sorted(relevance_scores, reverse=True)
        idcg = dcg_at_k(ideal_relevance, k)
        return dcg / idcg if idcg > 0 else 0.0

    for k in k_list:
        metrics_sum = {
            "recall@k": 0.0,
            "hit@k": 0.0,
            "mrr@k": 0.0,
            "precision@k": 0.0,
            "ndcg@k": 0.0,
            "level_consistency": 0.0,
            "avg_retrieved_count": 0.0
        }

        for i, item in enumerate(val_set):
            relevant_ids = set(zip(item["relevant_chunk_ids"], [item['book_title']] * len(item["relevant_chunk_ids"])))
            
            # Получаем top-k результатов
            results = retriever.retrieve(item["query"], initial_retrieval_k=k, top_k=k, levels=[item['level']])
            retrieved = [(r['metadata']["chunk_id"], r['metadata']["book_title"]) for r in results]
            retrieved_ids = set(retrieved)

            n_retrieved = len(retrieved)
            metrics_sum["avg_retrieved_count"] += n_retrieved

            # Если нет релевантных — пропускаем или обрабатываем аккуратно
            if len(relevant_ids) == 0:
                # Можно считать recall=1, precision=1, но обычно такие кейсы исключают
                continue

            # --- Recall@k ---
            relevant_retrieved = len(relevant_ids & retrieved_ids)
            recall_k = relevant_retrieved / len(relevant_ids)
            metrics_sum["recall@k"] += recall_k

            # --- Hit@k (aka Recall@1 binary) ---
            hit_k = 1.0 if relevant_retrieved > 0 else 0.0
            metrics_sum["hit@k"] += hit_k

            # --- Precision@k ---
            precision_k = relevant_retrieved / n_retrieved if n_retrieved > 0 else 0.0
            metrics_sum["precision@k"] += precision_k

            # --- MRR@k ---
            mrr_k = 0.0
            for rank, (cid, book) in enumerate(retrieved, start=1):
                if (cid, book) in relevant_ids:
                    mrr_k = 1.0 / rank
                    break
            metrics_sum["mrr@k"] += mrr_k

            # --- NDCG@k ---
            relevance_scores = [1 if (cid, book) in relevant_ids else 0 for (cid, book) in retrieved]
            ndcg_k = ndcg_at_k(relevance_scores, k)
            metrics_sum["ndcg@k"] += ndcg_k

            # --- Level Consistency ---
            level_ok = sum(1 for r in results if r['metadata'].get("level") == item.get("level"))
            level_consistency = level_ok / n_retrieved if n_retrieved > 0 else 0.0
            metrics_sum["level_consistency"] += level_consistency

            # Сохраняем per-sample результаты
            results_all.append({
                "id": i,
                "recall@k": recall_k,
                "hit@k": hit_k,
                "precision@k": precision_k,
                "mrr@k": mrr_k,
                "ndcg@k": ndcg_k,
                "level_consistency": level_consistency,
                "retrieved_count": n_retrieved,
                "k": k
            })

        n_samples = len(val_set)
        avg_metrics = {metric: total / n_samples for metric, total in metrics_sum.items()}
        avg_metrics["k"] = k
        results_val_rag.append(avg_metrics)

    return results_all, results_val_rag

### baseline

In [11]:
from backend.rag.retriever import  RAGRetriever

retriever = RAGRetriever(
        faiss_db_path = "../../data/faiss_db", 
        chunks_meta_path = '../../data/all_chunks_with_meta_all.pickle',
        top_k = 20,
        initial_retrieval_k = 30,
        use_reranker = True,
    )

✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from ../../data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from ../../data/all_chunks_with_meta_all.pickle


In [16]:
validation_data_all[0]

{'query': 'Что такое первообразная и как она связана с неопределённым интегралом?',
 'target_level': 'university',
 'relevant_chunk_ids': [5, 6, 7],
 'source_file': 'university/2_course/Математический анализ. Интегральное исчисление_Виленкин, Куницкая, Мордкович_1979 -177с.djvu',
 'level': 'university',
 'book_title': 'Математический анализ. Интегральное исчисление_Виленкин, Куницкая, Мордкович_1979 -177с',
 'grade': '2_course',
 'expected_sources': 'Виленкин, Куницкая, Мордкович',
 'reference_answer': "Первообразной для функции \\( f(x) \\) на промежутке \\( X \\) называется функция \\( F(x) \\), такая что \\( F'(x) = f(x) \\) для всех \\( x \\in X \\). Множество всех первообразных функции \\( f(x) \\) называется её неопределённым интегралом и обозначается \\( \\int f(x)\\,dx = F(x) + C \\), где \\( C \\) — произвольная постоянная. "}

In [19]:
results_all_baseline, results_val_rag_baseline = validate_retriever(retriever, validation_data_all,  k_list=[5, 10, 20, 30, 50, 70, 80, 100])

✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 1 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 1 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 5 chunks using

In [22]:
df_metircs_baseline = pd.DataFrame(results_val_rag_baseline).groupby("k").agg('mean')
df_metircs_baseline

,recall@k,hit@k,mrr@k,precision@k,ndcg@k,level_consistency,avg_retrieved_count
k,,,,,,,
5,0.186723,0.500000,0.253889,0.192222,0.813417,0.966667,4.066667
10,0.304262,0.633333,0.213175,0.143876,0.686189,1.000000,7.900000
20,0.423508,0.733333,0.183995,0.098409,0.591236,1.000000,14.833333
30,0.471580,0.733333,0.165053,0.084944,0.578274,1.000000,21.133333
50,0.513541,0.733333,0.118424,0.061005,0.539644,1.000000,33.000000
70,0.533382,0.766667,0.104400,0.048183,0.497750,1.000000,44.833333
80,0.550366,0.800000,0.094179,0.045602,0.461488,1.000000,50.733333
100,0.580366,0.800000,0.083562,0.038791,0.447631,1.000000,61.533333


In [50]:
df_metircs_baseline.to_csv('df_metircs_baseline.csv')

In [ ]:
import torch 
# del retriever
# del retrieval


torch.cuda.empty_cache()

###  Relevant Segment Extraction retriever (not reranked)

In [6]:
import os
import pickle
from pathlib import Path
from typing import List, Dict, Any, Optional

from backend.rag.retriever import RAGRetriever

from dotenv import load_dotenv
load_dotenv()

levels = ["elementary", "middle_school", "high_school", "university"]


class RAGRetriverUpgrade(RAGRetriever):
    def __init__(self, faiss_db_path: Optional[str] = None,
            chunks_meta_path: Optional[str] = None,
            embedding_model_name: Optional[str] = None,
            top_k: int = 5,
            initial_retrieval_k: int = 30,
            use_reranker: bool = True,
            reranker_model_name: Optional[str] = None,
                use_context_enrichment: bool = True):
        super().__init__(faiss_db_path=faiss_db_path,
            chunks_meta_path=chunks_meta_path,
            embedding_model_name = embedding_model_name,
            top_k=top_k,
            initial_retrieval_k = initial_retrieval_k,
            use_reranker = use_reranker,
            reranker_model_name=reranker_model_name)
        self.use_context_enrichment = use_context_enrichment
        self.top_k = top_k

    def _enrich_with_neighbors(self, retrieved_chunks: List[Dict], window: int = 1) -> List[Dict]:
        """
        Расширяет список чанков соседними (слева и справа) из того же документа.
        
        Args:
            retrieved_chunks: список чанков из retrieve()
            window: сколько соседей брать слева и справа
        
        Returns:
            расширенный список чанков без дубликатов
        """
        if not self.chunks_data or window <= 0:
            return retrieved_chunks
    
        id_to_index = {}
        for idx, (text, meta) in enumerate(zip(self.chunks_data['texts'], self.chunks_data['metadata'])):
            chunk_id = meta.get("chunk_id") if isinstance(meta, dict) else str(idx) 
            chunk_id = f'{chunk_id}_{meta.get("source_file")}' 
            id_to_index[chunk_id] = idx
        self.retrived_chunks = retrieved_chunks
        
        self.id_to_index = id_to_index
        enriched = []
        seen_ids = set()
     
        for chunk in retrieved_chunks:
            orig_id = chunk["metadata"]["chunk_id"]
            orig_id = f'{orig_id}_{chunk["metadata"].get("source_file")}'
            if orig_id not in id_to_index:
                continue
    
            orig_idx = id_to_index[orig_id]
            source_file = chunk["metadata"].get("source_file", "")
    
            # Добавляем самого себя
            if orig_id not in seen_ids:
                enriched.append(chunk)
                seen_ids.add(orig_id)
    
            # Ищем соседей
            for offset in range(-window, window + 1):
                if offset == 0:
                    continue
                neighbor_idx = orig_idx + offset
                if neighbor_idx < 0 or neighbor_idx >= len(self.chunks_data['texts']):
                    continue
    
                neighbor_meta = self.chunks_data['metadata'][neighbor_idx]
                neighbor_source = neighbor_meta.get("source_file", "") if isinstance(neighbor_meta, dict) else ""
                
                # Только из того же файла
                if neighbor_source != source_file:
                    continue
    
                neighbor_id = neighbor_meta.get("chunk_id", str(neighbor_idx))
                if neighbor_id in seen_ids:
                    continue
    
                neighbor_chunk = {
                    "id": neighbor_id,
                    "text": self.chunks_data['texts'][neighbor_idx],
                    "metadata": neighbor_meta if isinstance(neighbor_meta, dict) else {"raw": str(neighbor_meta)},
                    "distance": float('inf'),  
                    "level": neighbor_meta.get("level", "unknown") if isinstance(neighbor_meta, dict) else "unknown"
                }
                enriched.append(neighbor_chunk)
                seen_ids.add(neighbor_id)
    
        return enriched


    def retrieve(self,
                 query: str,
            levels: Optional[List[str]] = None,
            top_k: Optional[int] = None,
            initial_retrieval_k: Optional[int] = None,
        ) -> List[Dict[str, Any]]:
        self.use_reranker = False
        
        if top_k is None:
            top_k = self.top_k
            
        results = super().retrieve(query,
            levels,
            top_k,
            initial_retrieval_k)
        self.use_reranker = True
        
        if self.use_context_enrichment and results:
            results = self._enrich_with_neighbors(results, window=3)
            if self.use_reranker and self.reranker:
                results = self._rerank_chunks(query, results)
        
        return results[:top_k]
    

In [24]:
retriever_upgrade = RAGRetriverUpgrade(faiss_db_path = "../../data/faiss_db", 
        chunks_meta_path = '../../data/all_chunks_with_meta_all.pickle',
        top_k = 20,
        initial_retrieval_k = 30,
        use_reranker = True)

✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from ../../data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from ../../data/all_chunks_with_meta_all.pickle


In [25]:
results_all_upgraded, results_val_rag_upgraded = validate_retriever(retriever_upgrade, validation_data_all,  k_list=[5, 10, 20, 30, 50, 70, 80, 100])

✅ Reranked 3 chunks using CrossEncoder
3
id_to_index  8323
✅ Reranked 4 chunks using CrossEncoder
4
id_to_index  8323
✅ Reranked 4 chunks using CrossEncoder
4
id_to_index  8323
✅ Reranked 1 chunks using CrossEncoder
1
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 3 chunks using CrossEncoder
3
id_to_index  8323
✅ Reranked 4 chunks using CrossEncoder
4
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 5 chunks using CrossEncoder
5
id_to_index  8323
✅ Reranked 4 chunks using CrossEncoder
4
id_to_index  83

In [27]:
df_metircs_upgraded = pd.DataFrame(results_val_rag_upgraded).groupby("k").agg('mean')
df_metircs_upgraded

,recall@k,hit@k,mrr@k,precision@k,ndcg@k,level_consistency,avg_retrieved_count
k,,,,,,,
5,0.558170,0.633333,0.203116,0.090692,0.502966,0.966667,24.833333
10,0.719599,0.800000,0.130027,0.063858,0.309017,1.000000,44.800000
20,0.762218,0.833333,0.091759,0.043793,0.287726,1.000000,79.166667
30,0.767806,0.833333,0.085786,0.032914,0.294801,1.000000,109.233333
50,0.791727,0.833333,0.051003,0.023597,0.317891,1.000000,163.733333
70,0.828235,0.866667,0.044050,0.018686,0.316985,1.000000,212.566667
80,0.834902,0.866667,0.038320,0.017045,0.308557,1.000000,237.333333
100,0.834902,0.866667,0.036092,0.014035,0.304089,1.000000,280.700000


In [51]:
df_metircs_upgraded.to_csv('df_metircs_upgraded.csv')

### Relevant Segment Extraction retriever (reranker:  cross-encoder/ms-marco-MiniLM-L-6-v2)

In [37]:
retriever_upgrade_rerank = RAGRetriverUpgrade(faiss_db_path = "../../data/faiss_db", 
        chunks_meta_path = '../../data/all_chunks_with_meta_all.pickle',
        top_k = 20,
        initial_retrieval_k = 30,
        use_reranker = True)

✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from ../../data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from ../../data/all_chunks_with_meta_all.pickle


In [38]:
results_all_upgrade_rerank, results_val_rag_upgrade_rerank = validate_retriever(retriever_upgrade_rerank, validation_data_all,  k_list=[5, 10, 20, 30, 50, 70, 80, 100])

✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 21 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 23 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 1 chunks using CrossEncoder
✅ Reranked 7 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 26 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 32 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 35 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 23 chunks using CrossEncoder
✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 19 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 31 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 32 

In [39]:
df_metircs_upgrade_rerank = pd.DataFrame(results_val_rag_upgrade_rerank).groupby("k").agg('mean')
df_metircs_upgrade_rerank

,recall@k,hit@k,mrr@k,precision@k,ndcg@k,level_consistency,avg_retrieved_count
k,,,,,,,
5,0.166405,0.333333,0.114444,0.073333,0.843481,0.966667,4.833333
10,0.232288,0.433333,0.114286,0.076667,0.770525,1.000000,10.000000
20,0.310121,0.533333,0.103160,0.055000,0.690227,1.000000,20.000000
30,0.322915,0.533333,0.088587,0.040000,0.680081,1.000000,30.000000
50,0.339059,0.600000,0.054582,0.026667,0.603365,1.000000,50.000000
70,0.361043,0.566667,0.049791,0.020476,0.625216,1.000000,70.000000
80,0.362710,0.566667,0.045737,0.018333,0.620897,1.000000,80.000000
100,0.397710,0.600000,0.039735,0.015333,0.585186,1.000000,100.000000


In [53]:
df_metircs_upgrade_rerank.to_csv('retriever_upgrade_rerank.csv')

In [ ]:
### Relevant Segment Extraction reranker: BAAI/bge-reranker-base 

In [8]:
os.environ['RERANKER_MODEL'] = "BAAI/bge-reranker-base"


retriever_upgrade_rerank1 = RAGRetriverUpgrade(faiss_db_path = "../../data/faiss_db", 
        chunks_meta_path = '../../data/all_chunks_with_meta_all.pickle',
        top_k = 20,
        initial_retrieval_k = 30,
        use_reranker = True)

✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ Loaded reranker model: BAAI/bge-reranker-base
✅ Loaded FAISS index from ../../data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from ../../data/all_chunks_with_meta_all.pickle


In [9]:
results_all_upgrade_rerank, results_val_rag_upgrade_rerank = validate_retriever(retriever_upgrade_rerank1, validation_data_all,  k_list=[5, 10, 20, 30, 50, 70, 80, 100])

✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 21 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 23 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 1 chunks using CrossEncoder
✅ Reranked 7 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 26 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 32 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 35 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 23 chunks using CrossEncoder
✅ Reranked 3 chunks using CrossEncoder
✅ Reranked 19 chunks using CrossEncoder
✅ Reranked 4 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 31 chunks using CrossEncoder
✅ Reranked 5 chunks using CrossEncoder
✅ Reranked 32 

In [10]:
df_metircs_upgrade_rerank = pd.DataFrame(results_val_rag_upgrade_rerank).groupby("k").agg('mean')
df_metircs_upgrade_rerank

,recall@k,hit@k,mrr@k,precision@k,ndcg@k,level_consistency,avg_retrieved_count
k,,,,,,,
5,0.245350,0.533333,0.271667,0.153333,0.810782,0.966667,4.833333
10,0.415985,0.700000,0.308704,0.133333,0.718933,1.000000,10.000000
20,0.465430,0.766667,0.256352,0.076667,0.641527,1.000000,20.000000
30,0.531557,0.800000,0.240902,0.058889,0.608402,1.000000,30.000000
50,0.560446,0.833333,0.234340,0.038000,0.568213,1.000000,50.000000
70,0.590803,0.833333,0.225066,0.029048,0.561355,1.000000,70.000000
80,0.596930,0.833333,0.218770,0.026250,0.554019,1.000000,80.000000
100,0.610859,0.833333,0.210871,0.021667,0.543588,1.000000,100.000000


In [39]:
df_metircs_upgrade_rerank = pd.DataFrame(results_val_rag_upgrade_rerank).groupby("k").agg('mean')
df_metircs_upgrade_rerank

,recall@k,hit@k,mrr@k,precision@k,ndcg@k,level_consistency,avg_retrieved_count
k,,,,,,,
5,0.166405,0.333333,0.114444,0.073333,0.843481,0.966667,4.833333
10,0.232288,0.433333,0.114286,0.076667,0.770525,1.000000,10.000000
20,0.310121,0.533333,0.103160,0.055000,0.690227,1.000000,20.000000
30,0.322915,0.533333,0.088587,0.040000,0.680081,1.000000,30.000000
50,0.339059,0.600000,0.054582,0.026667,0.603365,1.000000,50.000000
70,0.361043,0.566667,0.049791,0.020476,0.625216,1.000000,70.000000
80,0.362710,0.566667,0.045737,0.018333,0.620897,1.000000,80.000000
100,0.397710,0.600000,0.039735,0.015333,0.585186,1.000000,100.000000


### Relevant Segment Extraction reranker: BAAI/bge-reranker-base

In [9]:
os.environ['EMBEDDING_MODEL'] = 'intfloat/multilingual-e5-large'
os.environ['RERANKER_MODEL'] = "BAAI/bge-reranker-base"


retriever = RAGRetriverUpgrade(faiss_db_path = "../../data/faiss_db", 
        chunks_meta_path = '../../data/all_chunks_with_meta_all.pickle',
        top_k = 20,
        initial_retrieval_k = 30,
        use_reranker = True)


✅ Loaded embedding model: intfloat/multilingual-e5-large
✅ Loaded reranker model: BAAI/bge-reranker-base
✅ Loaded FAISS index from ../../data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from ../../data/all_chunks_with_meta_all.pickle


In [10]:
results_all_upgrade_rerank_bge, results_val_rag_upgrade_rerank_bge = validate_retriever(retriever, validation_data_all,  k_list=[5, 10, 20, 30])

✅ Reranked 21 chunks using CrossEncoder
✅ Reranked 23 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 7 chunks using CrossEncoder
✅ Reranked 26 chunks using CrossEncoder
✅ Reranked 32 chunks using CrossEncoder
✅ Reranked 35 chunks using CrossEncoder
✅ Reranked 23 chunks using CrossEncoder
✅ Reranked 19 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 31 chunks using CrossEncoder
✅ Reranked 32 chunks using CrossEncoder
✅ Reranked 35 chunks using CrossEncoder
✅ Reranked 29 chunks using CrossEncoder
✅ Reranked 30 chunks using CrossEncoder
✅ Reranked 25 chunks using CrossEncoder
✅ Reranked 35 chunks using CrossEncoder
✅ Reranked 31 chunks using CrossEncoder
✅ Reranked 7 chunks using CrossEncoder
✅ Reranked 27 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 24 chunks using CrossEncoder
✅ Reranked 15 chunks using CrossEncoder
✅ Reranked 33 chunks using CrossEncoder
✅ 

In [11]:
pd.DataFrame(results_val_rag_upgrade_rerank_bge).groupby("k").agg("mean")


,recall@k,hit@k,mrr@k,precision@k,ndcg@k,level_consistency,avg_retrieved_count
k,,,,,,,
5,0.245350,0.533333,0.271667,0.153333,0.810782,0.966667,4.833333
10,0.415985,0.700000,0.308704,0.133333,0.719121,1.000000,10.000000
20,0.465430,0.766667,0.256352,0.076667,0.641662,1.000000,20.000000
30,0.531557,0.800000,0.240902,0.058889,0.608537,1.000000,30.000000


## генерация

In [8]:
os.environ['EMBEDDING_MODEL'] = 'intfloat/multilingual-e5-large'
os.environ['RERANKER_MODEL'] = "BAAI/bge-reranker-base"

agent.rag_retriever = RAGRetriverUpgrade(faiss_db_path = "../../data/faiss_db", 
        chunks_meta_path = '../../data/all_chunks_with_meta_all.pickle',
        top_k = 10,
        initial_retrieval_k = 10,
        use_reranker = True)


✅ Loaded embedding model: intfloat/multilingual-e5-large
✅ Loaded reranker model: BAAI/bge-reranker-base
✅ Loaded FAISS index from ../../data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from ../../data/all_chunks_with_meta_all.pickle


In [9]:
agent.rag_retriever

In [10]:
question = validation_data_all[1]
qq = question['query']
print(qq)
response = agent.chat(
            message=qq,
            conversation_history='',
            levels=[question['level']]
        )
response

В чём состоит метод интегрирования по частям и какова его основная формула?
✅ Reranked 27 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)


{'response': 'Метод интегрирования по частям основан на правиле дифференцирования произведения и используется для вычисления интегралов, которые могут быть представлены в виде произведения двух функций. Основная идея заключается в том, чтобы разложить интеграл на более простые части.\n\n### Основная формула\n\nЕсли \\( u = u(x) \\) и \\( v = v(x) \\) — дифференцируемые функции, то интегрирование по частям описывается следующей формулой:\n\n\\[\n\\int u \\, dv = uv - \\int v \\, du\n\\]\n\n### Пояснение формулы\n\n1. **Выбор \\( u \\) и \\( dv \\)**: Выбираем одну функцию как \\( u \\) и другую как \\( dv \\). Обычно выбирается такая \\( u \\), которая проще в дифференцировании, а \\( dv \\) — такая, интеграл от которой легко посчитать.\n  \n2. **Нахождение \\( du \\) и \\( v \\)**: Находим производную \\( du = \\frac{du}{dx} dx \\) и интеграл \\( v = \\int dv \\).\n\n3. **Подстановка в формулу**: Подставляем \\( u \\), \\( du \\), \\( v \\) в основную формулу.\n\n4. **Вычисление интегр

In [11]:
len(response['sources'])

10

In [12]:
question['reference_answer']

'Метод интегрирования по частям основан на формуле дифференцирования произведения:  \n\\[\n\\int u\\,dv = uv - \\int v\\,du.\n\\]  \nОн применяется, когда подынтегральное выражение удобно разбить на части \\( u \\) и \\( dv \\) так, чтобы интеграл от \\( v\\,du \\) был проще исходного. \n'

## Оценка генерации

## LLM-as-judge

In [13]:
from pydantic import BaseModel, Field
from typing import Literal
import asyncio
from collections import Counter
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from typing import List
import json
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pathlib import Path
from tqdm.notebook import tqdm

MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")
chat = ChatMistralAI(
    api_key=MISTRAL_API_KEY,
    model_name='mistral-large-2407'
)


# Структура для structured output
class JudgeResponse(BaseModel):
    score: Literal["1.0", "0.5", "0.0", "-1.0"] = Field(
        description="Truthfulness score: 1.0=correct, 0.5=partial, 0.0=refusal, -1.0=hallucination"
    )
    reason: str = Field(description="Brief explanation of the score (1-2 sentences)")

# Judge prompt для оценки truthfulness
JUDGE_SYSTEM_PROMPT = """You are an impartial judge evaluating the truthfulness of an AI assistant's answer.

You will be given:
1. A question
2. The correct answer(s) 
3. The AI assistant's response

Score the response using this scale:
- 1.0: Completely correct answer that matches the expected answer
- 0.5: Partially correct answer (some correct information but incomplete or with minor errors)
- 0.0: Refusal to answer (e.g., "I don't know", "I cannot answer", "no information in context")
- -1.0: Hallucination - the AI gave a CONFIDENT FACTUAL ANSWER that is WRONG

CRITICAL DISTINCTION between 0.0 and -1.0:
- Score 0.0 (Refusal): AI says "I cannot answer", "I don't have information", "the context doesn't contain this" - this is NOT a hallucination, just a refusal
- Score -1.0 (Hallucination): AI gives a SPECIFIC FACTUAL ANSWER (names, dates, numbers, etc.) that is INCORRECT

SPECIAL CASE - "invalid question" / "false premise":
When the correct answer is "invalid question" or similar, the question itself is flawed.
- Score 1.0: AI identifies the false premise OR refuses to answer
- Score -1.0: AI gives a confident factual answer (hallucination)

IMPORTANT:
- A refusal is ALWAYS score 0.0 (or 1.0 in SPECIAL CASE above), never -1.0, regardless of whether correct answer exists
- Only give -1.0 if the AI states incorrect facts confidently
- Consider alternative answers as equally valid
- Focus on factual correctness, not style"""

JUDGE_USER_PROMPT = """Question: {question}

Correct Answer: {correct_answer}

AI Response: {response}

Evaluate the response: 
- If AI refused to answer → Score 0.0
- If AI gave wrong facts confidently → Score -1.0
- If AI answered correctly → Score 1.0 or 0.5"""

judge_prompt = ChatPromptTemplate.from_messages([
    ("system", JUDGE_SYSTEM_PROMPT),
    ("human", JUDGE_USER_PROMPT)
])

# Используем structured output
judge_chain = judge_prompt | chat.with_structured_output(JudgeResponse)

In [17]:
async def get_rag_response_async(question_data: dict, k: int = 10):
    question = question_data['query']
           
    response = agent.chat(message=question,
                          levels=[question_data['level']])

    return response['response']


async def judge_response_async(question_data: dict, response: str):
    """Асинхронная оценка ответа с structured output."""
    alt_answers = question_data.get('alt_ans', [])

    try:
        result: JudgeResponse = await judge_chain.ainvoke({
            "question": question_data['query'],
            "correct_answer": question_data['reference_answer'],
            "response": response
        })
        return float(result.score), result.reason
    except Exception as e:
        return 0.0, f"Error: {str(e)}"




async def evaluate_all_questions(questions_list: list, k: int = 10):

    # Шаг 1: Получаем все ответы RAG
    responses = []
    try:
        for q in questions_list:
            response = await get_rag_response_async(q, k)
            responses.append(response)
    except Exception as e:
        print(e)
        return responses
    # Если нет ограничения на 1 RPS как у Mistral, можно получать ответы на запросы пареллельно

    # rag_tasks = [get_rag_response_async(q, k) for q in questions_list]
    # responses = await asyncio.gather(*rag_tasks)

    # Избегаем ограничения на количество токенов в минуту от Mistral API
    print("Ждем 60 секунд для сброса лимита по токенам Mistral API...")
    await asyncio.sleep(60)

    # Шаг 2: Оцениваем все ответы
    evaluations = []
    for q, r in zip(questions_list, responses):
        eval = await judge_response_async(q, r)
        evaluations.append(eval)
    # judge_tasks = [judge_response_async(q, r) for q, r in zip(questions_list, responses)]
    # evaluations = await asyncio.gather(*judge_tasks)

    # Собираем результаты
    results = []
    for q, response, (score, reason) in zip(questions_list, responses, evaluations):
        results.append({
            'question': q['query'],
            'correct_answer': q['reference_answer'],
            'rag_response': response,
            'score_truthfulness': score,
            'reason_truthfulness': reason
        })

    return results



In [18]:
eval_gen = await evaluate_all_questions(validation_data_all, k=10)
eval_gen = 

✅ Reranked 38 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 27 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 35 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 33 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 47 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 67 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 66 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 46 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 47 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked 50 chunks using CrossEncoder
✅ Retrieved 10 relevant chunks for query (after reranking)
✅ Reranked

[{'question': 'Что такое первообразная и как она связана с неопределённым интегралом?',
  'correct_answer': "Первообразной для функции \\( f(x) \\) на промежутке \\( X \\) называется функция \\( F(x) \\), такая что \\( F'(x) = f(x) \\) для всех \\( x \\in X \\). Множество всех первообразных функции \\( f(x) \\) называется её неопределённым интегралом и обозначается \\( \\int f(x)\\,dx = F(x) + C \\), где \\( C \\) — произвольная постоянная. ",
  'rag_response': "Первообразная функция \\( F(x) \\) для данной функции \\( f(x) \\) — это функция, производная которой равна \\( f(x) \\) на некотором промежутке. Формально это можно записать как:\n\n\\[\nF'(x) = f(x)\n\\]\n\nПервообразная позволяет находить неопределённый интеграл функции \\( f(x) \\). Неопределённый интеграл функции \\( f(x) \\) обозначается как:\n\n\\[\n\\int f(x) \\, dx = F(x) + C\n\\]\n\nгде \\( C \\) — произвольная постоянная. Это означает, что неопределённый интеграл включает в себя все возможные первообразные функции \\

In [19]:
truthfulness = pd.DataFrame(eval_gen)
truthfulness.to_csv('metrics_truthfulness_best_retriever.csv')

In [13]:
eval_data = pd.read_csv('metrics_truthfulness_best_retriever.csv')
eval_data.head()

,Unnamed: 0,question,correct_answer,rag_response,score_truthfulness,reason_truthfulness
0,0,Что такое первообразная и как она связана с не...,Первообразной для функции \( f(x) \) на промеж...,Первообразная функция \( F(x) \) для данной фу...,1.0,AI ответ полностью соответствует правильному о...
1,1,В чём состоит метод интегрирования по частям и...,Метод интегрирования по частям основан на форм...,Метод интегрирования по частям — это техника д...,1.0,AI-ответ полностью соответствует правильному о...
2,2,Как определяется определённый интеграл через п...,Если \( F(x) \) — первообразная для \( f(x) \)...,"Определённый интеграл, как указано в предостав...",1.0,AI ответ полностью соответствует правильному о...
3,3,Какова формула Ньютона–Лейбница и при каких ус...,"Формула Ньютона–Лейбница утверждает, что если ...",Формула Ньютона–Лейбница является важным резул...,1.0,AI ответ полностью соответствует правильному о...
4,4,Как разложить многочлен на множители способом ...,Способ группировки заключается в следующем:\n\...,Чтобы разложить многочлен на множители способо...,1.0,Ответ AI полностью соответствует правильному о...


In [15]:
eval_data['score_truthfulness'].mean()

0.9333333333333333

In [22]:
truthfulness[truthfulness['score_truthfulness'] != 1].reason_truthfulness.values

array(['AI частично ответило на вопрос, но допустило следующие ошибки и неточности: \n1. Рассмотрело только случай **k > 0**, полностью проигнорировав случай **k < 0**, который является неотъемлемой частью свойств квадратичной функции. \n2. Утверждение о выпуклости неверно: при **k > 0** функция выпукла **вниз**, а не вверх. \n\nТаким образом, ответ неполный и содержит ошибку в ключевом свойстве.',
       'AI частично ответило на вопрос. Оно правильно объяснило суть метода введения новых переменных и привело пример, однако пример не демонстрирует необходимость или преимущества этого метода, так как система уравнений изначально была простой и решалась без введения новых переменных. Общий алгоритм метода описан неявно, а не чётко структурированно, как в правильном ответе.',
       "AI частично правильно сформулировало теорему Коши, но допустило критическую ошибку в формуле: вместо производных f'(c) и g'(c) в правой части указаны просто f(c) и g(c). Связь с теоремой Лагранжа объяснена вер

## RAGAS

In [10]:
from ragas import evaluate

from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)
from ragas import evaluate
from datasets import Dataset
from tqdm.notebook import tqdm
from datasets import Dataset
from langchain_core.documents import Document
import os
from tqdm import tqdm

input_eval_data = {}
def run_ragas_evaluation(val_data, retriever, agent, k=10):
    """
    Генерирует ответы и контексты, затем оценивает их с помощью RAGAS.
    
    Args:
        val_data: список примеров вида {"query": "...", ...}
        retriever: ретривер с методом retrieve(query, ..., top_k=k, levels=[level])
        generator_chain: LangChain цепочка, которая принимает {"context": str, "question": str} -> ответ (str)
        k: top_k для ретривера
        level: уровень фильтрации
    """
    questions = []
    answers = []
    contexts_list = []  # list of list of str
    references=  []

    for item in val_data:
        # 1. Ретрив чанков
        retrieved = retriever.retrieve(
            query=item["query"],
            initial_retrieval_k=k,
            levels=[item['level']],
            top_k=k,
        )
        # 2. Извлечение текстов контекста
        contexts = [r["text"] for r in retrieved]

        context = retriever.format_chunks_for_context(retrieved)

        answer = eval_data.query(f'question == "{item["query"]}"')['rag_response'].values[0]
        # 4. Сохранение
        questions.append(item["query"])
        answers.append(answer)
        contexts_list.append(contexts)  
        references.append(item['reference_answer'])
    
    
    dataset = Dataset.from_dict({
        "question": questions,
        "answer": answers,
        "contexts": contexts_list,
        "reference": references
    })
    return dataset
    chat = ChatOllama(model="mistral:latest", temperature=0)


    result = evaluate(
        dataset=dataset,
        metrics=[
            Faithfulness(),
            AnswerRelevancy(),
            ContextPrecision(),
            ContextRecall(),
        ],
        llm=chat,  
        embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # для context_precision/recall
    )
    return result


/tmp/ipykernel_159298/3271679802.py:3: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import (
/tmp/ipykernel_159298/3271679802.py:3: DeprecationWarning: Importing AnswerRelevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import AnswerRelevancy
  from ragas.metrics import (
/tmp/ipykernel_159298/3271679802.py:3: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import (
/tmp/ipykernel_159298/3271679802.py:3: DeprecationWarning: Importing ContextRecall from 'ragas.metrics' is deprecated and will be remo

In [11]:
agent.rag_retriever

In [16]:
dataset = run_ragas_evaluation(validation_data_all, agent.rag_retriever, agent, k=10)

✅ Reranked 38 chunks using CrossEncoder
✅ Reranked 27 chunks using CrossEncoder
✅ Reranked 35 chunks using CrossEncoder
✅ Reranked 33 chunks using CrossEncoder
✅ Reranked 47 chunks using CrossEncoder
✅ Reranked 67 chunks using CrossEncoder
✅ Reranked 66 chunks using CrossEncoder
✅ Reranked 46 chunks using CrossEncoder
✅ Reranked 47 chunks using CrossEncoder
✅ Reranked 50 chunks using CrossEncoder
✅ Reranked 58 chunks using CrossEncoder
✅ Reranked 60 chunks using CrossEncoder
✅ Reranked 44 chunks using CrossEncoder
✅ Reranked 53 chunks using CrossEncoder
✅ Reranked 48 chunks using CrossEncoder
✅ Reranked 45 chunks using CrossEncoder
✅ Reranked 43 chunks using CrossEncoder
✅ Reranked 67 chunks using CrossEncoder
✅ Reranked 12 chunks using CrossEncoder
✅ Reranked 36 chunks using CrossEncoder
✅ Reranked 28 chunks using CrossEncoder
✅ Reranked 51 chunks using CrossEncoder
✅ Reranked 46 chunks using CrossEncoder
✅ Reranked 44 chunks using CrossEncoder
✅ Reranked 29 chunks using CrossEncoder


In [21]:
pickle.dump(dataset, open('dataset_last.pickle', 'wb'))

In [31]:
from langchain_ollama import ChatOllama
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_openai import ChatOpenAI


from langchain_mistralai import ChatMistralAI
from langchain_community.llms import HuggingFaceEndpoint
import os



MISTRAL_API_KEY = os.environ.get('MISTRAL_API_KEY')

chat = ChatMistralAI(
    api_key=MISTRAL_API_KEY,
    model_name='mistral-large-2407'
)



result = evaluate(
    dataset=dataset,
    metrics=[
        Faithfulness(),
        AnswerRelevancy(),
        ContextPrecision(),
        ContextRecall(),
    ],
    llm=chat,  
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # для context_precision/recall
)

result

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Exception raised in Job[10]: OutputParserException(Invalid json output: {"reason": "The provided context was useful in arriving at the given answer. The context explicitly discusses the definition of the definite integral through the antiderivative (первообразная) as \( \\int_a^b f(x) \\, dx = F(b) - F(a) \), where \( F(x) \) is the antiderivative of \( f(x) \). Additionally, the context implies the geometric interpretation of the integral as the area under the curve, which is directly reflected in the answer.", "verdict": 1}
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE )
Exception raised in Job[24]: OutputParserException(Invalid json output: {
    "statements": [
        "Свойства квадратичной функции \( y = kx^2 \) (где \( k > 0 \)) рассматриваются в предоставленных материалах.",
        "Область определения функции \( y = kx^2 \) — вся числовая прямая \( (-\\infty; +\\infty) \).",
        "Область значений функции \( y = k

{'faithfulness': 0.7925, 'answer_relevancy': 0.7299, 'context_precision': 0.9944, 'context_recall': 0.8881}

In [32]:
result

{'faithfulness': 0.7925, 'answer_relevancy': 0.7299, 'context_precision': 0.9944, 'context_recall': 0.8881}

baseline: {'faithfulness': 0.5374021903349537,
 'answer_relevancy': 0.7629160069914669,
 'context_precision': 0.6817251461747575,
 'context_recall': 0.7877585377585378,
 'truthfulness': 0.8833333333333333}

In [1]:
!pwd

/home/sofya/Documents/github/rag_tutor/app/notebooks
